In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


1. Elegir una hora en el formato de 24hrs, eg. 18:30 para las 6:30 pm y 05:00 para las 5 am, por la cual filtraremos los escenarios 
2. Elegir la acción a efectuar on/off para prender/apagar el trigger

Nota: esta acción no distingue entre procesos (BBU, R1, 100t, etc.), por lo cual toma todos los escenarios existentes en Dataiku.

In [62]:
hour = '07:00'
action = 'off'

In [63]:
import dataiku
from dataiku import api_client
import pandas as pd

In [64]:
client = dataiku.api_client() # Connect to dataiku
project = client.get_project('project_name') #Take project
all_scenarios = pd.DataFrame(project.list_scenarios()) #Take ALL scenaries from the project as a DataFrame

scenarios_triggered = all_scenarios[all_scenarios.triggerDigest == 'Every day at {}'.format(hour)]  #Time filter
scenarios_triggered_ids = scenarios_triggered['id'] #Lista de ids

for i,scenario_trigg_id in enumerate(scenarios_triggered_ids):
    #Loop for turning on/off the triggers
    scenario = project.get_scenario(scenario_trigg_id)
    
    ##### Take the scenario name
    last_run_finished=scenario.get_last_finished_run()
    details = last_run_finished.get_details()
    nombre=details['stepRuns'][0]['scenarioRun']['scenario']['params']['steps'][0]['params']['config']['parameters']['table-name']
    
    ##### Take setting and turn on/off the trigger
    settings = scenario.get_settings()
    try:
        if action == 'on':
            settings.active = True #Turn off the main trigger 
            print('El escenario {} se activo correctamente'.format(nombre), "{} de {}".format(i+1, len(scenarios_triggered_ids)))
        elif action == 'off':
            settings.active = False #Turn off the main trigger 
            print('El escenario {} se desactivo correctamente'.format(nombre),"{} de {}".format(i+1, len(scenarios_triggered_ids)))
            
        settings.save() #Save settings changes
    except:
        raise Exception("Algo salió mal en el escenario {}".format(nombre))


('El escenario ra_cust_trx_line_gl_dist_all se desactivo correctamente', '1 de 3')
('El escenario xla_distribution_links se desactivo correctamente', '2 de 3')
('El escenario GL_IMPORT_REFERENCES se desactivo correctamente', '3 de 3')
